<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Addestramento_modello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [183]:
#Librerie
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from tensorflow.keras import layers,models,losses,optimizers,applications
import numpy as np
from art.utils import load_dataset

##Def variabili

In [ ]:
#salvataggio modello
model_save_name = 'model_DenseNet201'
path_model_save = F"/content/gdrive/My Drive/ModelliCNN/{model_save_name}" 

##Collegamento google drive

In [ ]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**Dataset**

##Load dataset stl10

In [184]:
(x_train, y_train), (x_test_tot, y_test_tot), min_, max_ = load_dataset(str("stl10")) #carica train e test set

##Riduzione dataset

In [185]:
#x_test, y_test = x_test_tot[:500], y_test_tot[:500]

#nessuna riduzione test set
x_test, y_test = x_test_tot, y_test_tot

##Stampa dimensioni

In [186]:
im_shape = x_train[0].shape
print("dimensioni immagine: ",im_shape)
print("dimensioni train set: ",x_train.shape)
print("dimensioni vettore classi reali: ",y_train.shape)
print("dimensioni test set: ",x_test.shape)

dimensioni immagine:  (96, 96, 3)
dimensioni train set:  (5000, 96, 96, 3)
dimensioni vettore classi reali:  (5000, 10)
dimensioni test set:  (8000, 96, 96, 3)


# **Creazione modello**

##Load model pre-trained

In [187]:
#creazione rete usando vgg16 preaddestrata e aggiungendo gli ultimi livelli per adattarla al problema di 10 classi
from tensorflow.keras import applications as applications

#importa rete vgg16 addestrata sul dataset imagenet, esclude gli ultimi livelli e come input pongo dimensioni 96,96,3 
model_conv = applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(im_shape[0],im_shape[1],im_shape[2]))

# Freeze all the layers (non modifico i pesi dei livelli inferiori)
for layer in model_conv.layers[:]:
    layer.trainable = False

Instructions for updating:
Colocations handled automatically by placer.
74850304/74836368 [==============================] - 1s 0us/step


# **Training**
> per riaddestrare eseguire solo questa sezione



##Aggiunta classificatore

In [189]:
# creo un modello aggiungendo livelli alla rete importata
model = models.Sequential()
# Add the vgg convolutional base model
model.add(model_conv)
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax')) #10: numero di valori in uscita (le classi)

#stampa info del modello
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 3, 3, 1920)        18321984  
_________________________________________________________________
flatten (Flatten)            (None, 17280)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              17695744  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 36,027,978
Trainable params: 17,705,994
Non-trainable params: 18,321,984
_________________________________________________________________


##Configura parametri modello

In [191]:
# Configure the model for training (setta i parametri del modello)
model.compile(
        loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"]
    )

##Addestramento

In [193]:
history= model.fit(x_train,y_train,epochs=15,batch_size=128,validation_split=0.2)

Train on 4000 samples, validate on 1000 samples
Epoch 1/15
4000/4000 [==============================] - ETA: 0s - loss: 1.8159 - accuracy: 0.5095

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


4000/4000 [==============================] - 22s 5ms/sample - loss: 1.8159 - accuracy: 0.5095 - val_loss: 0.8665 - val_accuracy: 0.7050
Epoch 2/15
4000/4000 [==============================] - 11s 3ms/sample - loss: 0.7559 - accuracy: 0.7372 - val_loss: 0.7929 - val_accuracy: 0.7370
Epoch 3/15
4000/4000 [==============================] - 11s 3ms/sample - loss: 0.5091 - accuracy: 0.8213 - val_loss: 0.7413 - val_accuracy: 0.7450
Epoch 4/15
4000/4000 [==============================] - 11s 3ms/sample - loss: 0.3709 - accuracy: 0.8715 - val_loss: 0.7260 - val_accuracy: 0.7720
Epoch 5/15
4000/4000 [==============================] - 11s 3ms/sample - loss: 0.2587 - accuracy: 0.9172 - val_loss: 0.7384 - val_accuracy: 0.7650
Epoch 6/15
4000/4000 [==============================] - 11s 3ms/sample - loss: 0.2073 - accuracy: 0.9367 - val_loss: 0.7394 - val_accuracy: 0.7750
Epoch 7/15
4000/4000 [==============================] - 11s 3ms/sample - loss: 0.1546 - accuracy: 0.9567 - val_loss: 0.7329 - val

KeyboardInterrupt: ignored

#**Testing**

In [196]:
value_preds=model.predict(x_test) #contiene i valori tra 0 e 1 predetti per ognuna delle 10 classi e per ogni immagine
preds = np.argmax(value_preds, axis=1) #(le predizioni vanno da 0 a 9 e indicano la classe predetta)
acc = (np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0])*100
print('\033[1m'+"Accuracy on test set:", (round(acc,2)))

Accuracy on test set: 76.24


#**Salvataggio modello**

In [ ]:
#salva modello su drive
model.save(path_model_save)